# Building a Simple LLM Application with LangChain Expression Language (LCEL)

This quickstart guides you through building a simple LLM application using LangChain Expression Language (LCEL). Our application will perform a straightforward English-to-French text translation. While this is a relatively simple use case, it effectively demonstrates the power of LCEL for chaining components.

You'll get a high-level overview of:

* **Using Language Models**: Interacting with both OpenAI and Groq (for open-source models like Gemma).
* **Prompt Templates and Output Parsers**: Structuring LLM inputs and extracting clean outputs.
* **LangChain Expression Language (LCEL)**: Chaining components together using the intuitive `|` operator.
* **Debugging and Tracing with LangSmith**: Observing the execution flow of your application.
* **Deploying with LangServe (Conceptual)**: Understanding how to serve your LangChain applications as APIs.

## Key Concepts:

* **LangChain Expression Language (LCEL)**: A declarative way to compose "Runnables" (LangChain's building blocks) into chains using a concise syntax (e.g., `component1 | component2`). LCEL offers benefits like streaming, async support, parallel execution, and easier integration with LangSmith and LangServe.
* **`ChatPromptTemplate`**: A LangChain component for creating structured prompts, especially useful for chat models, allowing the definition of system, user, and assistant messages.
* **`StrOutputParser`**: A simple LangChain output parser that extracts the string content from an LLM's response.
* **Groq**: A fast inference engine that provides access to open-source LLMs (like Gemma, Llama 3) at extremely high speeds via an API.
* **LangSmith**: An observability platform for LLM applications that provides detailed traces of your chain's execution, aiding in debugging and performance analysis.
* **LangServe**: A library for deploying LangChain applications as REST APIs, making your LLM apps accessible via HTTP endpoints.

## Setup: API Keys and Environment Variables

Ensure your `.env` file is configured with `OPENAI_API_KEY`, `GROQ_API_KEY`, `LANGCHAIN_API_KEY`, and `LANGCHAIN_PROJECT`.

### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file.
load_dotenv()

# Set OpenAI API key from environment variables.
# This is necessary for using OpenAI models.
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# Set Groq API key from environment variables.
# This is necessary for using Groq's fast inference models.
groq_api_key = os.getenv("GROQ_API_KEY")

# Display a part of the Groq API key to confirm it's loaded (for debugging, don't expose full key).
# print(f"Groq API Key (first 5 chars): {groq_api_key[:5]}*****")


# --- LangSmith Tracking Setup ---
# Set LangChain API key for LangSmith.
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
# Enable LangSmith tracing. "true" automatically sends traces to LangSmith.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# Define the project name under which traces will be organized in LangSmith.
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# Import ChatOpenAI for OpenAI models and ChatGroq for Groq models.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Initialize a ChatGroq model using 'Gemma2-9b-It'.
# Groq provides incredibly fast inference for open-source models.
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

# Display the initialized model object.
print("--- Initialized Groq Model ---")
print(model)

/Users/pro/python-learning/Machine Learning/15-LCEL/lcel-env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


--- Initialized Groq Model ---
client=<groq.resources.chat.completions.Completions object at 0x1100f4f10> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x110101670> model_name='Gemma2-9b-It' model_kwargs={} groq_api_key=SecretStr('**********')


### Using Language Models Directly with Messages

LLMs often work best with a list of "messages" that define roles (System, Human, AI) to provide context and instructions. 

In [4]:
# Import message types from langchain_core.messages
from langchain_core.messages import HumanMessage, SystemMessage

# Define a list of messages.
# SystemMessage provides instructions or context to the AI.
# HumanMessage provides the user's input.
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

# Invoke the model directly with the list of messages.
# The 'invoke' method sends the messages to the LLM and returns its response.
result = model.invoke(messages)

# Print the entire response object received from the LLM.
# This object contains not just the content but also metadata.
print("--- Direct Model Invocation Result ---")
print(result)

--- Direct Model Invocation Result ---
content="Hello - Bonjour\n\nHow are you? - Comment allez-vous ? \n\n(More informal) How are you? - Ça va ?\n\n\nLet me know if you'd like to know more translations!\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 21, 'total_tokens': 66, 'completion_time': 0.081818182, 'prompt_time': 0.003343189, 'queue_time': 0.249786541, 'total_time': 0.085161371}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--65af6a67-09fd-4054-b3fb-27f69ad9dff7-0' usage_metadata={'input_tokens': 21, 'output_tokens': 45, 'total_tokens': 66}


### Parsing Output with StrOutputParser

The raw `result` from `model.invoke` is a complex object. Often, we only need the generated text content. `StrOutputParser` extracts this content as a simple string.

In [5]:
# Import StrOutputParser to extract the string content from LLM responses.
from langchain_core.output_parsers import StrOutputParser

# Initialize the string output parser.
parser = StrOutputParser()

# Invoke the parser with the raw LLM result to get just the string content.
parsed_result = parser.invoke(result)

# Print the parsed string result.
print("--- Parsed Output (String) ---")
print(parsed_result)

--- Parsed Output (String) ---
Hello - Bonjour

How are you? - Comment allez-vous ? 

(More informal) How are you? - Ça va ?


Let me know if you'd like to know more translations!



### Using LCEL: Chaining Components

LangChain Expression Language (LCEL) allows you to chain Runnables together using the `|` (pipe) operator, making your code concise and readable. The output of one component becomes the input of the next.

In [6]:
# Create a simple chain using LCEL: model | parser.
# This means the messages go to the model, and the model's output then goes to the parser.
chain = model | parser

# Invoke the chain with the same list of messages.
# The result will directly be the parsed string output.
chained_result = chain.invoke(messages)

# Print the result from the chained components.
print("--- Result from Chained Components (model | parser) ---")
print(chained_result)

--- Result from Chained Components (model | parser) ---
Here are a couple of ways to say "Hello, How are you?" in French:

* **Bonjour, comment allez-vous ?** (Formal)
* **Salut, comment vas-tu ?** (Informal) 


Let me know if you'd like to see other variations! 😊



### Prompt Templates for Dynamic Inputs

`ChatPromptTemplate` allows you to create dynamic prompts with placeholders, making your applications flexible. Instead of hardcoding messages, you can define a template and fill in variables at runtime.

In [7]:
# Import ChatPromptTemplate for creating dynamic prompts.
from langchain_core.prompts import ChatPromptTemplate

# Define a generic system template with a placeholder for the target language.
generic_template = "Translate the following into {language}:"

# Create a ChatPromptTemplate from messages.
# The system message uses the generic_template.
# The user message has a placeholder "{text}" for the content to be translated.
prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

# Invoke the prompt template with specific values for the placeholders.
# The `invoke` method returns a `PromptValue` object, which can be converted to messages.
result_prompt = prompt.invoke({"language": "French", "text": "Hello"})

# Convert the PromptValue object to a list of messages.
# This shows how the template is filled out into actual messages.
print("--- Prompt Template Result (Converted to Messages) ---")
print(result_prompt.to_messages())

--- Prompt Template Result (Converted to Messages) ---
[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]


### Chaining Together Components with LCEL (Full Translation Application)

Now, let's put it all together: `PromptTemplate | Model | OutputParser`. This forms a complete, simple LLM application that takes dynamic inputs, processes them with the LLM, and provides a clean string output.

In [8]:
## Chaining together components with LCEL
# Create the full chain: prompt | model | parser.
# 1. 'prompt' formats the input into messages.
# 2. 'model' receives the messages and generates a response.
# 3. 'parser' extracts the string content from the model's response.
chain = prompt | model | parser

# Invoke the full chain with a dictionary containing values for the prompt's placeholders.
# The result is the final translated string.
final_translation = chain.invoke({"language": "French", "text": "How are you doing today?"})

# Print the final translated output.
print("--- Final Translated Output from Full Chain ---")
print(final_translation)

--- Final Translated Output from Full Chain ---
Here are a few ways to translate "How are you doing today?" into French:

* **Comment vas-tu aujourd'hui ?** (This is the most common and informal way to ask.)
* **Comment allez-vous aujourd'hui ?** (This is a more formal way to ask.)
* **Ça va aujourd'hui ?** (This is a very casual way to ask.)



Let me know if you'd like to see more variations!



## LangSmith and LangServe Overview

### Debugging and Tracing Your Application with LangSmith

Since you've set up `LANGCHAIN_TRACING_V2="true"` and your `LANGCHAIN_API_KEY` and `LANGCHAIN_PROJECT` environment variables, every `invoke` call on your `chain` automatically sends a trace to LangSmith.

**To check your usage and traces in LangSmith:**

1.  **Go to the LangSmith UI**: Open your web browser and navigate to [smith.langchain.com](https://smith.langchain.com/).
2.  **Log in**: Use the credentials you signed up with.
3.  **Navigate to your Project**: On the left sidebar, click on "Projects". You should see a project listed with the name you set in your `.env` file (e.g., `my-translation-app`).
4.  **View Runs**: Click on your project name. You will see a list of "Runs" (each invocation of your `chain` or `model`).
5.  **Explore a Trace**: Click on any individual run. You'll see a detailed "trace" showing each step of your chain (e.g., how the prompt was formatted, the LLM call, and the parsing step). You can inspect inputs, outputs, latency, and token usage for each component. This is incredibly powerful for debugging and optimizing your LLM applications.

### Deploying Your Application with LangServe (Conceptual)

LangServe makes it easy to deploy your LangChain `Runnable`s (like our `chain`) as a REST API. This allows other applications or users to interact with your LLM application via HTTP requests.

While we are not doing a full deployment in this quickstart, conceptually, if you had this chain defined in a Python script (e.g., `app.py`), you could expose it using LangServe:

```python
# # Example of what an app.py for LangServe might look like
# from fastapi import FastAPI
# from langserve import add_routes
#
# # Assuming 'chain' is defined as in this notebook
# # from your_module import chain
#
# app = FastAPI(
#     title="Translation App",
#     version="1.0",
#     description="A simple API for English to French translation."
# )
#
# add_routes(app, chain, path="/translate")
#
# # To run this, you would save it as app.py and run:
# # uvicorn app:app --reload